In [2]:
import sys
sys.path.append('..')
from theano import function, config, shared, sandbox
import theano.tensor as T
import numpy
import time
import theano_lstm
import numpy as np

In [21]:
%%time
reload(theano_lstm)
config.mode="FAST_RUN"
hidden_size = 40
network_opt = theano_lstm.lstm_rnn(300,
                  [(hidden_size, hidden_size+1)],
                  20, log_dir='test_log')
# network_opt.initialize_training_adadelta()

CPU times: user 1.16 s, sys: 24 ms, total: 1.18 s
Wall time: 1.18 s


In [9]:
num_examples = 9
input_seq = np.random.normal(size=(2200, 300, num_examples)).astype('float32')
seq_lengths = [np.random.randint(low=500,high=2200) for x in range(num_examples)]
answers = np.zeros(shape=(50, num_examples)).astype('float32')
answers[10, :] = 1

In [12]:
input_seq.shape

(2200, 300, 9)

In [22]:
network_opt.process(input_seq, seq_lengths)

ValueError: GpuElemwise. Input dimension mis-match. Input 1 (indices start at 0) has shape[1] == 9, but the output's size on that axis is 40.
Apply node that caused the error: GpuElemwise{mul,no_inplace}(<CudaNdarrayType(float32, row)>, GpuElemwise{Composite{(scalar_sigmoid((i0 + i1)) * tanh(i2))},no_inplace}.0)
Inputs types: [CudaNdarrayType(float32, row), CudaNdarrayType(float32, matrix)]

HINT: Use another linker then the c linker to have the inputs shapes and strides printed.
HINT: Re-running with most Theano optimization disabled could give you a back-trace of when this node was created. This can be done with by setting the Theano flag 'optimizer=fast_compile'. If that does not work, Theano optimizations can be disabled with 'optimizer=None'.
HINT: Use the Theano flag 'exception_verbosity=high' for a debugprint and storage map footprint of this apply node.
Apply node that caused the error: forall_inplace,gpu,scan_fn}(Shape_i{0}.0, GpuSubtensor{int64:int64:int8}.0, GpuAlloc{memset_